In [29]:
# import
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

## I. Text Small Analysis

In [6]:
# Open The text file
with open("input.txt","r", encoding = "utf-8") as f:
    text = f.read()
print(text[:100])
print(f'the text has a length of {len(text)} characters')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


1115394

## II. Tokenizer and Encoder/Decoder

Tokenize : Transform Text --> Integers

Short size of vocabulary but Long sequence length : Character-level Tokenizer
Simple Tokeinizer : with a mapping of characters to integers using the position in chars (The one we will use)

Long size of vocabulary but Short sequence length : sub-word/Word-level Tokenizer
Google Tokenizer : SentencePiece : sub-words units level tokenizer
OPENAI : Tiktoken : BPE Tokenizer


In [7]:
# Characters of the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Encoder : Transform text --> Vectors/integers $\newline$
Decoder : Transform Vectors/integers --> text

In [11]:
# Create the mapping from characters to integer
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
def encode(s): return [stoi[c] for c in s]
def decode(L): return "".join([itos[i] for i in L])

print(encode("je suis Osiris"))
print(decode([48, 43, 1, 57, 59, 47, 57, 1, 27, 57, 47, 56, 47, 57]))

[48, 43, 1, 57, 59, 47, 57, 1, 27, 57, 47, 56, 47, 57]
je suis Osiris


## Using **PyTorch Tensors** to store text as **embeddings**

- **Ease of use**: All modules (**nn.Embedding**, **nn.Linear**, **Transformer**, etc.) expect **torch.Tensor**.  

- **GPU/TPU**: Once it’s a **tensor**, it can be moved to **GPU/TPU** with `x.to("cuda")` or `x.to("tpu")`. **NumPy** only works on **CPU**.  

- **Backward / Autograd**: **PyTorch tensors** keep track of operations for **automatic gradient computation**.  

  **Example**:  
  ```python
  import torch

  x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
  y = x * 2
  y.backward(torch.ones_like(x))
  print(x.grad)  # x remembers its expression with respect to y allowing it to compute x.grad automatically

    ```
- `x` remembers its expression with respect to `y`, which allows computing **x.grad** automatically.  

- **Integrated tensors** in **PyTorch** (with **optimizers** like **Adam**, **SGD**, **loss functions**, training on **mini-batches**).  

- **PyTorch** is optimized in **C++/CUDA** → ultra-fast **matrix computations**.  


In [14]:
data = torch.tensor(encode(text))
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [16]:
# Spliting train and test
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

## III. Chunking


**Chunk = split the text** into smaller pieces (e.g. paragraphs of **300–500 tokens**).

### 🔹 Why chunk?
- **Context window limit**: LLMs can only read a fixed number of tokens (2k, 4k, 32k...).$\newline$
--> Chunking helps fit large texts into the model's context window.$\newline$
--> Attention mechanisms in transformers have quadratic complexity with respect to input length, so shorter inputs are more efficient.$\newline$
- **Memory efficiency**: smaller chunks use less memory during processing.
- **Better local coherence**: smaller chunks keep the meaning clear.  
- **Efficient search (RAG)**: embeddings are stored per chunk, making retrieval precise.  
- **Performance**: faster similarity search + less dilution in attention.  

Rule of thumb: chunks of **200–500 tokens** usually work best.

In [17]:
# Exemple
block_size = 8
x = train_data[:block_size] # input of the transformer : first block size characters
y = train_data[1:block_size+1] # next block size character offset by one --> target for each pos
for t in range(block_size):
    context = x[:t+1] # all the tokens before the token at t
    target = y[t]     # expected token at t
    print(f'input: {context}, target: {y[t]}')


input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58,  1])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58,  1, 15])
input: <bound method Kernel.raw_input of 

## Batching

- **Batch** = a set of sequences processed in parallel on the GPU.  
- **Main purpose**: speed up training with GPU parallelization.  
- Each sequence in the batch is **independent**:  
  - The loss is calculated for each sequence, then combined to update the model weights.  
- Batching is used **only to process multiple sequences in parallel** for GPU efficiency, **without sharing context between them**.
- **Weight update**: after computing the losses for all the sequences in the batch, **the gradients are combined to update the model’s weights.**


In [26]:

batch_size = 4 # Number of sequences that will be processed in parallel
block_size = 8 # Maximum content length for prediction

def get_batch(split):
    # Generate small batch of data of inputs x and target y
    data = train_data if split =='train' else val_data
    # Choice of random offset (starting point) duri,g training --> Avoid overfitting (<== if we start always at the same point)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])       # context (stacked by row in a batch_size x block_size Tensor)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])   # target/prediction
    return x,y

# Visualisation
xb,yb = get_batch('train')
print('-------------Inputs-------------')
print(xb.shape)
print(xb)
print('-------------Targets-------------')
print(yb.shape)
print(yb)
print('---------------------------------')

-------------Inputs-------------
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-------------Targets-------------
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------


In [27]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"input = {context.tolist()}, target = {target}")

input = [24], target = 43
input = [24, 43], target = 58
input = [24, 43, 58], target = 5
input = [24, 43, 58, 5], target = 57
input = [24, 43, 58, 5, 57], target = 1
input = [24, 43, 58, 5, 57, 1], target = 46
input = [24, 43, 58, 5, 57, 1, 46], target = 43
input = [24, 43, 58, 5, 57, 1, 46, 43], target = 39
input = [44], target = 53
input = [44, 53], target = 56
input = [44, 53, 56], target = 1
input = [44, 53, 56, 1], target = 58
input = [44, 53, 56, 1, 58], target = 46
input = [44, 53, 56, 1, 58, 46], target = 39
input = [44, 53, 56, 1, 58, 46, 39], target = 58
input = [44, 53, 56, 1, 58, 46, 39, 58], target = 1
input = [52], target = 58
input = [52, 58], target = 1
input = [52, 58, 1], target = 58
input = [52, 58, 1, 58], target = 46
input = [52, 58, 1, 58, 46], target = 39
input = [52, 58, 1, 58, 46, 39], target = 58
input = [52, 58, 1, 58, 46, 39, 58], target = 1
input = [52, 58, 1, 58, 46, 39, 58, 1], target = 46
input = [25], target = 17
input = [25, 17], target = 27
input = [2

## IV. Language Models

### A. Bigram Model

## Bigram Model

- A **bigram model** is a very simple language model that predicts a token based **only on the previous token**.  
- **Unigram model** → assumes each token is independent.  
- **Bigram model** → probability of a token depends only on the one before it.  
- **N-gram model** → generalization where the token depends on the previous *n-1* tokens.  


### Formula

For a sequence of tokens $ w_1, w_2, \dots, w_n $:  

$$
P(w_1, w_2, \dots, w_n) \approx \prod_{i=2}^n P(w_i \mid w_{i-1})
$$


##  Reminder: Role of `nn.Embedding` in PyTorch

- **`nn.Embedding(num_embeddings, embedding_dim)`** = a **lookup table** (similar to dictionnary) that maps token indices to vectors.  

- Each token (represented by an **integer index**) is associated with a **learnable vector**.  

- In a **language model**:
  - Input: token indices (`idx`).
  - Output: embeddings (vectors) that represent each token.  

- Example with `nn.Embedding(vocab_size, vocab_size)` in a Bigram model:
  - Each token index is mapped to a **vector of size `vocab_size`**.
  - This vector acts as the **logits** (scores) for predicting the **next token**.  
  - Applying **softmax** on these logits gives a probability distribution over the vocabulary.  

➡️ In short: **`nn.Embedding` converts discrete tokens into continuous vectors** that the model can use for learning and prediction.


In [ ]:
class BigramLangaugeModels(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx, targets):

        logits = self.token_embedding_table(idx) # size (B,T,C)

        return logits


    